In [1]:
#!poetry install

In [2]:
# Update this for your data path.
instance_data_dir = "/Volumes/ml/datasets/test_datasets/single_image_dataset"
pretrained_model_name_or_path = "black-forest-labs/FLUX.1-dev"
# Your public model name after it's pushed to the hub.
hub_model_id = "simpletuner-lora"
tracker_project_name = "flux-training"

# Validation prompt
validation_prompt = "A photo-realistic image of a cat"

train_batch_size = 1
learning_rate = 1e-4

# choices: int8-quanto, fp8-quanto, no_change (mac and a100/h100 users get int4 and int2 as well)
base_model_precision = "no_change"

In [3]:
lycoris_config = {
    "algo": "lokr",
    "multiplier": 1.0,
    "linear_dim": 10000,
    "linear_alpha": 1,
    "factor": 12,
    "apply_preset": {
        "target_module": [
            "Attention",
            "FeedForward"
        ],
        "module_algo_map": {
            "Attention": {
                "factor": 12
            },
            "FeedForward": {
                "factor": 6
            }
        }
    }
}
# write to config/lycoris_config.json
import json
with open("config/lycoris_config.json", "w") as f:
    json.dump(lycoris_config, f)

In [4]:
training_config = {
    "mixed_precision":"bf16",
    "model_type":"lora",
    "pretrained_model_name_or_path":pretrained_model_name_or_path,
    "gradient_checkpointing":True,
    "cache_dir": "cache",
    "set_grads_to_none":True,
    "gradient_accumulation_steps":1,
    "resume_from_checkpoint":"latest",
    "snr_gamma":5,
    "num_train_epochs":0,
    "max_train_steps":10000,
    "metadata_update_interval":65,
    "optimizer":"adamw_bf16",
    "learning_rate":learning_rate,
    "lr_scheduler":"polynomial",
    "seed":42,
    "lr_warmup_steps":100,
    "output_dir":"output/models",
    "non_ema_revision": False,
    "aspect_bucket_rounding":2,
    "inference_scheduler_timestep_spacing":"trailing",
    "training_scheduler_timestep_spacing":"trailing",
    "report_to":"wandb",
    "lr_end":1e-8,
    "compress_disk_cache":True,
    "push_to_hub":True,
    "hub_model_id":hub_model_id,
    "push_checkpoints_to_hub":True,
    "model_family":"flux",
    "benchmark_base_model":True,
    "train_batch":train_batch_size,
    "max_workers":32,
    "read_batch_size":25,
    "write_batch_size":64,
    "caption_dropout_probability":0.1,
    "torch_num_threads":8,
    "image_processing_batch_size":32,
    "vae_batch_size":4,
    "validation_prompt":validation_prompt,
    "num_validation_images":1,
    "validation_num_inference_steps":20,
    "validation_seed":42,
    "minimum_image_size":0,
    "resolution":1024,
    "validation_resolution":"1024x1024",
    "resolution_type":"pixel_area",
    "lycoris_config":"config/lycoris_config.json",
    "lora_type":"lycoris",
    "base_model_precision":base_model_precision,
    "checkpointing_steps":500,
    "checkpoints_total_limit":5,
    "validation_steps":500,
    "tracker_run_name":hub_model_id,
    "tracker_project_name":tracker_project_name,
    "validation_guidance":3.0,
    "validation_guidance_real":1.0,
    "validation_guidance_rescale":0.0,
    "validation_negative_prompt":"blurry, cropped, ugly",
}
# write to config/config.json
with open("config/config.json", "w") as f:
    json.dump(training_config, f, indent=4)

In [5]:
dataloader_config = [
    {
        "id": "my-dataset-512",
        "type": "local",
        "instance_data_dir": instance_data_dir,
        "crop": False,
        "crop_style": "random",
        "minimum_image_size": 128,
        "resolution": 512,
        "resolution_type": "pixel_area",
        "repeats": "4",
        "metadata_backend": "discovery",
        "caption_strategy": "filename",
        "cache_dir_vae": "cache/vae-512"
    },
    {
        "id": "my-dataset-1024",
        "type": "local",
        "instance_data_dir": instance_data_dir,
        "crop": False,
        "crop_style": "random",
        "minimum_image_size": 128,
        "resolution": 1024,
        "resolution_type": "pixel_area",
        "repeats": "4",
        "metadata_backend": "discovery",
        "caption_strategy": "filename",
        "cache_dir_vae": "cache/vae-1024"
    },
    {
        "id": "my-dataset-512-crop",
        "type": "local",
        "instance_data_dir": instance_data_dir,
        "crop": False,
        "crop_style": "random",
        "minimum_image_size": 128,
        "resolution": 512,
        "resolution_type": "pixel_area",
        "repeats": "4",
        "metadata_backend": "discovery",
        "caption_strategy": "filename",
        "cache_dir_vae": "cache/vae-512-crop"
    },
    {
        "id": "my-dataset-1024-crop",
        "type": "local",
        "instance_data_dir": instance_data_dir,
        "crop": False,
        "crop_style": "random",
        "minimum_image_size": 128,
        "resolution": 1024,
        "resolution_type": "pixel_area",
        "repeats": "4",
        "metadata_backend": "discovery",
        "caption_strategy": "filename",
        "cache_dir_vae": "cache/vae-1024-crop"
    },
    {
        "id": "text-embed-cache",
        "dataset_type": "text_embeds",
        "default": True,
        "type": "local",
        "cache_dir": "cache/text"
    }
]
# write to config/multidatabackend.json
import json
with open("config/multidatabackend.json", "w") as f:
    json.dump(dataloader_config, f)

In [6]:
from helpers.training.trainer import Trainer
from helpers.training.state_tracker import StateTracker
from helpers import log_format
import logging
from os import environ

logger = logging.getLogger("SimpleTuner")
logger.setLevel(environ.get("SIMPLETUNER_LOG_LEVEL", "INFO"))

/Users/bghira/src/SimpleTuner/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from helpers.configuration.json_file import normalize_args
import os
os.environ['CONFIG_BACKEND'] = 'cmd'
os.environ['ENV'] = 'default'
StateTracker.set_config_path('config/')
loaded_config = normalize_args(training_config)

2024-08-31 20:56:05,034 [WARNING] (SimpleTuner) Skipping false argument: non_ema_revision


In [8]:
try:
    trainer = Trainer(loaded_config)
except Exception as e:
    import traceback
    logger.error(f"Failed to create Trainer: {e}, {traceback.format_exc()}")
    raise e

optimizer: {'precision': 'bf16', 'default_settings': {'betas': (0.9, 0.999), 'weight_decay': 0.01, 'eps': 1e-06}, 'class': <class 'helpers.training.optimizers.adamw_bfloat16.AdamWBF16'>}


2024-08-31 20:56:05,043 [WARNING] (ArgsParser) The VAE model madebyollin/sdxl-vae-fp16-fix is not compatible. Please use a compatible VAE to eliminate this warning. The baked-in VAE will be used, instead.
2024-08-31 20:56:05,043 [INFO] (ArgsParser) VAE Model: black-forest-labs/FLUX.1-dev
2024-08-31 20:56:05,044 [INFO] (ArgsParser) Default VAE Cache location: 
2024-08-31 20:56:05,044 [INFO] (ArgsParser) Text Cache location: cache
2024-08-31 20:56:05,045 [WARNING] (ArgsParser) Updating T5 XXL tokeniser max length to 512 for Flux.
2024-08-31 20:56:05,046 [WARNING] (ArgsParser) No data backend config provided. Using default config at config/multidatabackend.json.


Model family: flux


In [9]:
try:
    trainer.configure_webhook()
    trainer.init_noise_schedule()
    trainer.init_seed()

    trainer.init_huggingface_hub()
except Exception as e:
    logger.error(f"Failed to configure Trainer: {e}")
    raise e

2024-08-31 20:56:05,554 [INFO] (helpers.training.trainer) Logged into Hugging Face Hub as 'bghira'


In [10]:
try:
    trainer.init_preprocessing_models()
except Exception as e:
    logger.error(f"Failed to initialize preprocessing models: {e}")
    raise e

2024-08-31 20:56:05,559 [INFO] (helpers.training.trainer) Load VAE: black-forest-labs/FLUX.1-dev
2024-08-31 20:56:05,843 [INFO] (helpers.training.trainer) Loading VAE onto accelerator, converting from torch.float32 to torch.bfloat16
2024-08-31 20:56:05,952 [INFO] (helpers.training.trainer) Load tokenizers
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
2024-08-31 20:56:06,440 [INFO] (helpers.training.text_encoding) Loading OpenAI CLIP-L text encoder from black-forest-labs/FLUX.1-dev/text_encoder..
2024-08-31 20:56:06,683 [INFO] (helpers.training.text_encoding) Loading T5 XXL v1.1 text encoder from black-forest-labs/FLUX.1-dev/text_encoder_2..
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.30it/s]
2024-08-31 20:56:08,494 [INFO] (helpers.training.trainer) Moving text encoder to GPU.
2024-08-31 20:56:08,736 [INFO] (helpers.training.trainer) Moving text encoder 2 to GPU.


In [11]:
try:
    trainer.init_data_backend()
except Exception as e:
    logger.error(f"Failed to initialize data backend: {e}")
    raise e

2024-08-31 20:56:14,161 [INFO] (DataBackendFactory) Loading data backend config from config/multidatabackend.json
2024-08-31 20:56:14,162 [INFO] (DataBackendFactory) Configuring text embed backend: text-embed-cache
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00, 1132.49it/s]
2024-08-31 20:56:14,337 [INFO] (TextEmbeddingCache) (Rank: 0) (id=text-embed-cache) Listing all text embed cache entries
2024-08-31 20:56:14,339 [INFO] (DataBackendFactory) Pre-computing null embedding


In [ ]:
try:
    trainer.init_validation_prompts()
except Exception as e:
    logger.error(f"Failed to initialize validation prompts: {e}")
    raise e

In [ ]:
trainer.init_unload_text_encoder()

In [ ]:
trainer.init_unload_vae()

In [ ]:
trainer.init_load_base_model()

In [ ]:
trainer.init_precision()

In [ ]:
trainer.init_controlnet_model()

In [ ]:
trainer.init_freeze_models()

In [ ]:
trainer.init_trainable_peft_adapter()

In [ ]:
trainer.init_ema_model()

In [ ]:
trainer.init_validations()

In [ ]:
trainer.init_benchmark_base_model()

In [ ]:
trainer.init_optimizer()

In [ ]:
trainer.init_lr_scheduler()

In [ ]:
trainer.init_hooks()

In [ ]:
trainer.init_resume_checkpoint()

In [ ]:
trainer.init_post_load_freeze()

In [ ]:
trainer.init_prepare_models()

In [ ]:
trainer.init_trackers()

In [ ]:
trainer.train()